<a href="https://colab.research.google.com/github/manusmriti31/Learning_PyTorch/blob/main/Video%20notebooks/01_pytorch_workflow_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # PyToch Workflow
 Let's explore an example PyTorch end-to-end workflow.

Resources:
* Ground truth notebook - https://github.com/mrdbourke/pytorch-deep-learning/blob/main/01_pytorch_workflow.ipynb
* Book version of notebook - https://www.learnpytorch.io/01_pytorch_workflow/

 # What we are covering:
 1. Data (prepare and load)
 2. Build model
 3. Fitting the model to data (training)
 4. Making predictions and evaluating a model (inference)
 5. Saving and loading a model
 6. Putting it all together

In [ ]:
import torch
from torch import nn # nn contains all of PyTorch's building blocks for neural networks
import matplotlib.pyplot as plt

# Check PyTorch version
torch.__version__

'2.4.1+cu121'

## 1. Data (preparing and loading)

Data can be alomost anything... in machine learining
* Excel spreadsheet
* Images of any kind
* videos (YouTube has lots of data....)
* Audio like songs or podcasts
* DNA
* Text

Machine learinging is a game of two parts:
1. Get data into a numerical representation.
2. Build a model to learn patterns in that numerical representation

In [ ]:
f